In [37]:
import py_wgpu_fdm
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation


In [38]:
d = 0.5e-3                   # winding wire diameter (m)
D = 1.1e-3                   # core wire diameter (m)
alpha = 0.01                 # helix angle
L = 0.8
N = 6000


In [39]:
R = D/2
h = np.tan(alpha) * R

turns = L / h
turns_per_node = int(turns / N)
turns = N * turns_per_node

L = (int(turns) * h) #adjust L fornow, so we have a whole number of turns


p = 2 * np.pi * h
kappa = R / (R**2 + h**2) #curvature
tau = h / (R**2 + h**2) #torsion
S = L * np.sqrt(R**2 + h**2) / h

turns = L / h


In [40]:
dt = 1e-6
ds = turns_per_node * np.sqrt((2 * np.pi * R)**2 + p**2)

oversampling_factor = 256

ds

np.float64(0.0829421931298679)

In [41]:
def winding_properties(d, E, G, rho):

    r = d / 2
    A = np.pi * r**2
    I1 = I2 = (np.pi * r**4) / 4   # bending moments of inertia
    J = (np.pi * r**4) / 2         # polar moment of inertia for torsion

    # Radii of gyration
    k1 = np.sqrt(I1 / A)
    k2 = np.sqrt(I2 / A)
    kT = np.sqrt(J / A)

    # Flexural rigidities (bending about two axes) and torsional rigidity
    beta1 = E * I1
    beta2 = E * I2
    betaT = G * J

    # Shear rigidities (sigma1, sigma2) and extensional rigidity (sigma_p)
    sigma1 = G * A
    sigma2 = G * A
    sigma_p = E * A

    mass_per_length = rho * A

    return {
        'm': mass_per_length,
        'k': [k1, k2, kT],
        'beta': [beta1, beta2, betaT],
        'sigma': [sigma1, sigma2, sigma_p]
    }

In [42]:
loss = 1.0
rho = 8900
E=1.1
G=1.1

coil_props = winding_properties(d, E, G, rho)
print(coil_props)

{'m': 0.0017475109135593222, 'k': [np.float64(0.00012500000000000003), np.float64(0.00012500000000000003), np.float64(0.0001767766952966369)], 'beta': [3.3747577333484117e-15, 3.3747577333484117e-15, 6.749515466696823e-15], 'sigma': [2.1598449493429828e-07, 2.1598449493429828e-07, 2.1598449493429828e-07]}


In [43]:

# Create initial node data in local (u,v,w) helical frame
nodes = []
for i in range(N):
    position = [0.0, 0.0, 0.0]  # along tangent
    velocity = [0.0, 0.0, 0.0]
    angle = [0.0, 0.0, 0.0]
    angular_velocity = [0.0, 0.0, 0.0]
    
    nodes.append([position, velocity, angle, angular_velocity])



In [44]:
# Initialize simulation with given parameters
sim = py_wgpu_fdm.Simulation(
    nodes,
    oversampling_factor,
    dt / oversampling_factor,
    ds,
    loss,
    tau,
    kappa,
    coil_props['m'],
    5000,
    coil_props['beta'],
    [1, 1, 1],
    coil_props['k']
)

# Run the simulation for a number of frames
n_frames = 50
frames = []

for _ in range(n_frames):
    frame = sim.compute()
    frames.append(frame)


[py-wgpu-fdm/src/gpu_bindings.rs:226:9] &nodes_in.last() = Some(
    Node {
        positions: [
            0.0,
            0.0,
            0.0,
        ],
        _padding0: 0,
        velocities: [
            0.0,
            0.0,
            0.0,
        ],
        _padding1: 0,
        angles: [
            0.0,
            0.0,
            0.0,
        ],
        _padding2: 0,
        angular_velocites: [
            0.0,
            0.0,
            0.0,
        ],
        _padding3: 0,
        moments: [
            0.0,
            0.0,
            0.0,
        ],
        _padding4: 0,
        forces: [
            0.0,
            0.0,
            0.0,
        ],
        _padding5: 0,
    },
)


In [45]:
frames[-1][:40]

[[[0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0]],
 [[nan, 0.0, 0.0],
  [nan, 0.0, 0.0],
  [0.0, nan, 0.0],
  [0.0, nan, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0]],
 [[0.0, nan, nan],
  [0.0, nan, nan],
  [nan, 0.0, nan],
  [nan, 0.0, nan],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0]],
 [[nan, 0.0, 0.0],
  [nan, 0.0, 0.0],
  [0.0, nan, 0.0],
  [0.0, nan, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0]],
 [[0.0, nan, nan],
  [0.0, nan, nan],
  [nan, 0.0, nan],
  [nan, 0.0, nan],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0]],
 [[nan, 0.0, 0.0],
  [nan, 0.0, 0.0],
  [0.0, nan, 0.0],
  [0.0, nan, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0]],
 [[0.0, nan, nan],
  [0.0, nan, nan],
  [nan, 0.0, nan],
  [nan, 0.0, nan],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0]],
 [[nan, 0.0, 0.0],
  [nan, 0.0, 0.0],
  [0.0, nan, 0.0],
  [0.0, nan, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0]],
 [[0.0, nan, nan],
  [0.0, nan, nan],
  [nan, 0.0, nan],
  [nan, 0.0, nan],
  [0